### Import Packages and Set Directories

In [15]:
import os
import sys
import json
import inspect
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Patch
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from scipy.interpolate import make_interp_spline


# Set directories
notebook_dir = os.getcwd()
base_dir = os.path.join(notebook_dir, '..')
data_dir = os.path.join(base_dir, 'data')
stock_dir = os.path.join(base_dir, 'data', 'stocks')
src_dir = os.path.join(base_dir, 'src')
graph_dir = os.path.join(base_dir, 'results', 'graphs')
sys.path.append(src_dir)

# Import optimal portfolio
from optimal_portfolios import OptimalPortfolios
warnings.simplefilter(action='ignore', category=FutureWarning)

### Read in Data

In [2]:
stock_df = pd.read_csv(os.path.join(data_dir, 'stock_daily.csv'))

### Construct Optimal Portfolio

In [3]:
class ef_by_year():
    def __init__(self, stock_df, min_year, max_year, num_pfo, corr_threshold):
        self.stock_df = stock_df
        self.min_year = min_year
        self.max_year = max_year
        self.num_pfo = num_pfo
        self.corr_threshold = corr_threshold
        self._daily_return = None
        self._weekly_return = None
        self._monthly_return = None
        self._annual_return = None
        self._annual_variance = None
        self._weights = None
        
    def _calculate_ef(self):
        idx = 0
        for year in range(min_year, max_year+1):                    
            # Filter year data
            tmp_df = self.stock_df.copy()
            tmp_df['Date'] = pd.to_datetime(self.stock_df['Date'], format='%Y-%m-%d')
            year_df = tmp_df[tmp_df['Date'].dt.year == year].dropna(axis=1).set_index('Date')
            ef_year = OptimalPortfolios(year_df, self.num_pfo, self.corr_threshold)
            ef_year.calculate_efficient_frontier()
            
            # Extract return data from class
            ef_daily = ef_year.daily_return.reset_index()
            ef_weekly = ef_year.weekly_return.reset_index()
            ef_monthly = ef_year.monthly_return.reset_index()                        
            
            # Create header if starting 
            if idx == 0:
                ef_daily_combined = ef_daily.head(0)
                ef_weekly_combined = ef_weekly.head(0)
                ef_monthly_combined = ef_monthly.head(0)
                ef_annual_return = {}
                ef_annual_var = {}
                ef_weights = {}
            idx += 1            
            
            # Add to the dataframes and dictionaries
            ef_daily_combined = pd.concat([ef_daily_combined, ef_daily]).drop_duplicates(subset='Date', keep='last')
            ef_weekly_combined = pd.concat([ef_weekly_combined, ef_weekly]).drop_duplicates(subset='Year_Week', keep='last')
            ef_monthly_combined = pd.concat([ef_monthly_combined, ef_monthly]).drop_duplicates(subset='Year_Month', keep='last')
            ef_annual_return[year] = ef_year.pfo_returns
            ef_annual_var[year] = ef_year.pfo_vars
            ef_weights[year] = ef_year.weight_df
        
        # Convert decimal to percentage
        ef_daily_combined = ef_daily_combined.apply(lambda x: x*100 if x.name != 'Date' else x)
        ef_weekly_combined = ef_weekly_combined.apply(lambda x: x*100 if x.name != 'Year_Week' else x)
        ef_monthly_combined = ef_monthly_combined.apply(lambda x: x*100 if x.name != 'Year_Month' else x)
        
        # Set self return
        self._daily_return = ef_daily_combined
        self._weekly_return = ef_weekly_combined
        self._monthly_return = ef_monthly_combined
        self._annual_return = ef_annual_return
        self._annual_variance = ef_annual_var
        self._weights = ef_weights
    
    def convert_dict_to_df(self, year_dict):
        df = pd.DataFrame.from_dict(year_dict, orient='index')
        # Rename columns
        num_columns = len(df.columns)
        column_names = ['portfolio_' + str(i+1) for i in range(num_columns)]
        df.columns = column_names

        # Reset the index to get the 'Year' column
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'Year'}, inplace=True)
        return df
    
    @property
    def daily_return(self):
        if self._daily_return is None:
            self._calculate_ef()
        return self._daily_return

    @property
    def weekly_return(self):
        if self._weekly_return is None:
            self._calculate_ef()
        return self._weekly_return        

    @property
    def monthly_return(self):
        if self._monthly_return is None:
            self._calculate_ef()
        return self._monthly_return
    
    @property
    def annual_return(self):
        if self._annual_return is None:
            self._calculate_ef()
        self._annual_return_df = self.convert_dict_to_df(self._annual_return)
        return self._annual_return_df
        
    @property
    def annual_variance(self):
        if self._annual_variance is None:
            self._calculate_ef()
        self._annual_var_df = self.convert_dict_to_df(self._annual_variance)
        return self._annual_var_df

    @property
    def weights_dict(self):
        if self._weights is None:            
            self._calculate_ef()
        return self._weights

In [4]:
# Run the code to create optimal portfolios
min_year, max_year = 1994, 2023
num_pfo, corr_threshold = 5, 0.98
ef = ef_by_year(stock_df, min_year, max_year, num_pfo, corr_threshold)
display(ef.annual_return)

0 stocks removed, 661 stocks remain.
39 stocks removed, 670 stocks remain.
13 stocks removed, 735 stocks remain.
32 stocks removed, 750 stocks remain.
4 stocks removed, 827 stocks remain.
2 stocks removed, 857 stocks remain.
8 stocks removed, 887 stocks remain.
9 stocks removed, 911 stocks remain.
14 stocks removed, 926 stocks remain.
155 stocks removed, 806 stocks remain.
38 stocks removed, 941 stocks remain.
24 stocks removed, 987 stocks remain.
29 stocks removed, 1008 stocks remain.
28 stocks removed, 1039 stocks remain.
96 stocks removed, 1009 stocks remain.
118 stocks removed, 1001 stocks remain.
40 stocks removed, 1094 stocks remain.
46 stocks removed, 1118 stocks remain.
19 stocks removed, 1166 stocks remain.
178 stocks removed, 1039 stocks remain.
50 stocks removed, 1210 stocks remain.
14 stocks removed, 1284 stocks remain.
174 stocks removed, 1167 stocks remain.
119 stocks removed, 1245 stocks remain.
33 stocks removed, 1358 stocks remain.
69 stocks removed, 1347 stocks remain

,Year,portfolio_1,portfolio_2,portfolio_3,portfolio_4,portfolio_5
0,1994,0.029147,0.092192,0.180045,0.267889,0.355755
1,1995,0.300937,0.437552,0.574166,0.710779,0.847392
2,1996,0.222088,0.328450,0.434812,0.541172,0.647538
3,1997,0.343979,0.448529,0.565091,0.681652,0.798215
4,1998,0.010646,0.171379,0.332135,0.492888,0.653645
5,1999,-0.010062,0.159367,0.358855,0.558341,0.757839
6,2000,0.166270,0.336540,0.532957,0.729375,0.925794
7,2001,0.290178,0.327021,0.497994,0.668971,0.839945
8,2002,0.183785,0.183636,0.183508,0.236428,0.324734
9,2003,0.423475,0.481142,0.654978,0.828822,1.002659


In [26]:
# Save to csv
ef.annual_return.to_csv(os.path.join(data_dir, 'stocks/pfo_yearly.csv'), index=False)
ef.annual_variance.to_csv(os.path.join(data_dir, 'stocks/pfo_yearly_var.csv'), index=False)
ef.monthly_return.to_csv(os.path.join(data_dir, 'stocks/pfo_monthly.csv'), index=False)
ef.weekly_return.to_csv(os.path.join(data_dir, 'stocks/pfo_weekly.csv'), index=False)
ef.daily_return.to_csv(os.path.join(data_dir, 'stocks/pfo_daily.csv'), index=False)

# Save weights JSON
json_dict = {key: df.to_json() for key, df in ef.weights_dict.items()}
with open(os.path.join(data_dir, 'stocks/pfo_weights.json'), 'w') as json_file:
    json.dump(json_dict, json_file, indent=4)

### Read data example

In [25]:
# Read json file
with open(os.path.join(data_dir, 'stocks/pfo_weights.json'), 'r') as json_file:
    json_dict = json.load(json_file)

# Convert each JSON string back to a DataFrame
pfo_weights_dict = {key: pd.read_json(json_str) for key, json_str in json_dict.items()}
display(pfo_weights_dict['2000'])

,tickers,weight_1,weight_2,weight_3,weight_4,weight_5
0,AA,-6.351200e-06,-6.292400e-06,-6.820200e-06,-6.639000e-06,-6.460400e-06
1,AAON,6.321579e-03,5.530314e-03,4.797512e-03,4.146127e-03,3.043796e-03
2,AAPL,-1.253270e-05,-1.279160e-05,-1.254340e-05,-1.282700e-05,-1.157450e-05
3,ABM,1.823439e-03,2.907084e-03,3.269053e-03,3.123177e-03,2.296092e-03
4,ABT,-4.544000e-07,-1.186000e-07,-2.110000e-07,-6.160000e-08,2.213000e-07
...,...,...,...,...,...,...
882,TREX,-4.073500e-06,-4.534200e-06,-4.471900e-06,-5.418800e-06,-5.009500e-06
883,UPS,-4.654100e-06,-5.415300e-06,-6.178100e-06,-6.612400e-06,-6.059800e-06
884,UTHR,-3.432900e-06,-4.963000e-06,-6.131400e-06,-6.370000e-06,-6.240700e-06
885,WCC,-1.772500e-06,-3.602300e-06,-5.055500e-06,-5.425800e-06,-5.246300e-06
